# Main Visualization File
> Most of the inital code is ported from main.py

In [11]:
!pip3 install mmh3
!pip3 install torch

In [12]:
from table import Table
from sketches import Synopsis
import correlation
from RL_Agent import Autofeature_agent
from RL_Environment import ISOFAEnvironment
import pandas as pd
import numpy as np
import sketches
import os
from tqdm import tqdm

## Model Training

In [6]:
gcdata = "data/gc-data/"  # data directory
joinable = "DayLoc"  # feature that is joinable between tables
target = "Temperature"
# TODO: drop Date and Location on on the csv's

# define core table
print("Sketching Core Table...")
core_path = gcdata+"temp.csv"
t_core = Table(joinable, core_path)

y_feat = t_core.table[[joinable, target]]
t_core.table.drop([target], axis=1, inplace=True)
print(t_core.table.shape)

t_core.get_sketch()
core_syn = sketches.Synopsis(y_feat, attributes=[target], key=joinable) 
t_core.calc_corr_gain(core_syn)

Sketching Core Table...
(11959, 5)
Observed correlation: 0.9967236563887053
f0_ Temperature
Correlation bounds: 0.9914217299462094, 1.0023880394688534
Bootstrap 95% confidence interval for correlation: (0.9966099209170419, 0.9969456324960341)


/Users/korahughes/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and continuous values for target
  warnings.warn(msg, UserWarning)


Observed mutual info: 1.3934741864764607e-05
Observed correlation: 0.46844979302250733
f1_ Temperature
Correlation bounds: 0.35870779209686204, 0.6862245753802713
Bootstrap 95% confidence interval for correlation: (0.4417608372611969, 0.48294171228194777)


/Users/korahughes/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and continuous values for target
  warnings.warn(msg, UserWarning)


Observed mutual info: 1.5329966930149245e-06
Observed correlation: -0.04337375987667828
f2_ Temperature
Correlation bounds: -0.0560689785493934, 1
Bootstrap 95% confidence interval for correlation: (-0.06703062313611156, -0.019530919278972436)


/Users/korahughes/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and continuous values for target
  warnings.warn(msg, UserWarning)


Observed mutual info: 1.5582162168576519e-07
Observed correlation: -0.04306752506070069
f3_ Temperature
Correlation bounds: -0.06967795385932417, 1
Bootstrap 95% confidence interval for correlation: (-0.0660572668140623, -0.011198263063210055)


/Users/korahughes/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and continuous values for target
  warnings.warn(msg, UserWarning)


Observed mutual info: 2.2326064657877914e-07


### Instantiating Candidates

In [7]:
# define candidate tables
candidate_paths = [file for file in os.listdir(gcdata) if "temp" not in file]
t_candidates = []
for path in tqdm(candidate_paths):
    print("\n\nLooking at table", path)
    t_cand = Table(joinable, gcdata+path)
    # get rid of target variable in candidate table
    if target in t_cand.table.columns:
        t_cand.table.drop([target], axis=1, inplace=True)
    assert joinable in t_cand.table.columns, f"{joinable=} not found in {path}"
    # rename columns for less confusion on join
    renamer = dict([[col, path+'-'+col] for col in t_cand.table.columns if joinable not in col])
    t_cand.table = t_cand.table.rename(columns=renamer)
    
    # use synopsys for join estimation
    t_cand.get_sketch()  # ? sketch candidate table again
    # TODO: check missing values in join of core with voc_daily_summary.csv (nan vals in calc mutual info)
    t_cand.calc_corr_gain(core_syn)  # ? calculate correlation between candidate and itself
    # ? get feature-wise sketch
    t_cand.feature_scoring(20)
    for feat in t_core.df_sketch:
        if t_core.df_sketch[feat].dtype == 'object':
            t_core.df_sketch[feat] = t_core.df_sketch[feat].astype('category')
    for feat in t_cand.df_sketch:
        if t_cand.df_sketch[feat].dtype == 'object':
            t_cand.df_sketch[feat] = t_cand.df_sketch[feat].astype('category')
    t_candidates.append(t_cand)
    print('\n')

  0%|          | 0/9 [00:00<?, ?it/s]



Looking at table pm10_daily_summary.csv
Observed correlation: -0.04052077184093939
pm10_daily_summary.csv-o3_AQI Temperature
Correlation bounds: -0.11545603160451891, 0.06186103707872515
Bootstrap 95% confidence interval for correlation: (-0.06114979995508197, -0.026866817450155563)


/Users/korahughes/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and continuous values for target
  warnings.warn(msg, UserWarning)


Observed mutual info: -2.3883112706784242e-05
Observed correlation: -0.03914918161706282
pm10_daily_summary.csv-f0_ Temperature
Correlation bounds: -0.10866330292120387, 0.05937130428468072
Bootstrap 95% confidence interval for correlation: (-0.05528977419809841, -0.024862815696303654)


/Users/korahughes/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and continuous values for target
  warnings.warn(msg, UserWarning)


...

Observed mutual info: -4.727445097823487e-06
Observed correlation: 0.018518308778169266
no2_daily_summary.csv-f2_ Temperature
Correlation bounds: -0.03747019395393264, 1
Bootstrap 95% confidence interval for correlation: (-0.010693794043153347, 0.048091637360848934)


/Users/korahughes/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and continuous values for target
  warnings.warn(msg, UserWarning)


Observed mutual info: 0.0004304778805288345
Observed correlation: 0.017668130250040937
no2_daily_summary.csv-f3_ Temperature
Correlation bounds: -0.015686949104304362, 0.07565173598596169
Bootstrap 95% confidence interval for correlation: (-0.004569694339394844, 0.05384453530566701)


/Users/korahughes/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and continuous values for target
  warnings.warn(msg, UserWarning)
100%|██████████| 9/9 [05:37<00:00, 37.53s/it]

Observed mutual info: 0.00010612734488833767




### Actual Feature Selection

In [13]:
# instantiate model environment
model_target = 0
max_try_num = 7
# t_core.df_sketch.drop([target], axis=1, inplace=True)
print("\n\nDefining Environment")
env = ISOFAEnvironment(t_core, t_candidates, joinable, target, max_try_num)

# Parameters for the agent
learning_rate = 0.05
reward_decay = 0.9
e_greedy = 1
update_freq = 50
mem_cap = 1000
BDQN_batch_size = 3
print("Starting Training...")
autodata = Autofeature_agent(env, BDQN_batch_size, learning_rate, reward_decay, e_greedy, update_freq, mem_cap,
                                BDQN_batch_size)

print("\nAgent Ready!")

# Train the workload
auto_result = autodata.train_workload()



Defining Environment


--------------------Init:--------------------
Model R2 score: 99.81777340062351%
Train RMSE score: 0.35508748568598825
Model R2 score: 99.2143440951382%
Test RMSE Score: 1.4689548539323016
Starting Training...


AttributeError: partially initialized module 'torch._dynamo' has no attribute 'external_utils' (most likely due to a circular import)

### Storing Results

In [ ]:
res = pd.DataFrame(auto_result, columns=["RMSE", "Benefit", "Time", "Epsilon", "Features"])
res

### Loading Additional Results

In [29]:
data_df = pd.read_csv("information_rl.csv").drop("iteration", axis=1)
data_df["iteration"] = [i+1 for i in range(len(data_df.index))]
print(data_df.columns)
data_df

Index([' accuracy', 'iteration'], dtype='object')


,accuracy,iteration
0,0.527052,1
1,0.862500,2
2,0.862500,3
3,0.862500,4
4,0.862500,5
5,0.861007,6
6,0.887500,7
7,0.887500,8
8,0.875000,9
9,0.875000,10


In [32]:
df2 = pd.read_csv("results.csv").iloc[:, 1:]
df2["iteration"] = [i+1 for i in range(len(df2.index))]
print(df2.columns)
df2

Index(['rmse', 'benefit', 'time', 'epsilon', 'feats', 'iteration'], dtype='object')


,rmse,benefit,time,epsilon,feats,iteration
0,1.398555,0.070399,4.261874,1.000000,"['f0_', 'f1_', 'f2_', 'f3_', 'DayLoc', 'Temperature', 'no2_daily_summary.csv-f0_', 'no2_daily_summary.csv-f2_', 'rh_and_dp_daily_summary.csv-o3_AQI']",1
1,1.448778,0.020177,8.794105,1.000000,"['f0_', 'f1_', 'f2_', 'f3_', 'DayLoc', 'Temperature', 'voc_daily_summary.csv-f0_', 'o3_daily_summary.csv-f2_', 'o3_daily_summary.csv-f1_', 'wind_daily_summary.csv-o3_AQI']",2
2,1.435599,0.033355,13.604540,0.857696,"['f0_', 'f1_', 'f2_', 'f3_', 'DayLoc', 'Temperature', 'so2_daily_summary.csv-f0_', 'so2_daily_summary.csv-o3_AQI', 'pm10_daily_summary.csv-o3_AQI', 'so2_daily_summary.csv-f1_']",3
3,1.368567,0.100388,20.222984,0.735642,"['f0_', 'f1_', 'f2_', 'f3_', 'DayLoc', 'Temperature', 'so2_daily_summary.csv-f2_', 'wind_daily_summary.csv-f0_', 'wind_daily_summary.csv-f2_', 'wind_daily_summary.csv-f1_', 'so2_daily_summary.csv-f0_']",4
4,1.420885,0.048070,27.719471,0.630957,"['f0_', 'f1_', 'f2_', 'f3_', 'DayLoc', 'Temperature', 'no2_daily_summary.csv-f1_', 'no2_daily_summary.csv-f0_']",5
5,1.432176,0.036779,37.270393,0.541170,"['f0_', 'f1_', 'f2_', 'f3_', 'DayLoc', 'Temperature', 'wind_daily_summary.csv-f0_', 'wind_daily_summary.csv-o3_AQI']",6
6,1.433461,0.035494,45.983776,0.464159,"['f0_', 'f1_', 'f2_', 'f3_', 'DayLoc', 'Temperature', 'so2_daily_summary.csv-f1_', 'pm10_daily_summary.csv-f2_', 'o3_daily_summary.csv-f1_', 'voc_daily_summary.csv-f1_']",7
7,1.455624,0.013331,57.543370,0.398107,"['f0_', 'f1_', 'f2_', 'f3_', 'DayLoc', 'Temperature', 'pressure_daily_summary.csv-o3_AQI', 'pressure_daily_summary.csv-f3_', 'no2_daily_summary.csv-f3_']",8
8,1.383738,0.085217,65.884053,0.341455,"['f0_', 'f1_', 'f2_', 'f3_', 'DayLoc', 'Temperature', 'rh_and_dp_daily_summary.csv-f1_', 'o3_daily_summary.csv-f2_', 'rh_and_dp_daily_summary.csv-f3_']",9
9,1.401079,0.067875,75.175600,0.292864,"['f0_', 'f1_', 'f2_', 'f3_', 'DayLoc', 'Temperature', 'wind_daily_summary.csv-f2_', 'voc_daily_summary.csv-f0_', 'so2_daily_summary.csv-f0_', 'wind_daily_summary.csv-f1_']",10


## Visualizations

In [30]:
import plotly.express as px
import matplotlib.pyplot as plt

# rmse = [[row["accuracy"], i+1] for i, row in data_df.iterrows()]
# df = pd.Dataframe(data_df)
px.scatter(data_df, x="iteration", y=" accuracy")

In [36]:
px.scatter(df2, x="time", y="epsilon")